In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#git

In [61]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   FinRL_StockTrading_NeurIPS_2018.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [53]:
# !git add finrl/finrl_meta/env_stock_trading/env_test.py

!git add -u

In [54]:
!git config --global user.email "939778128@qq.com"
!git config --global user.name "EvanHong99"
!git commit -m "synchronize"

[master a55d7f1] synchronize
 2 files changed, 3 insertions(+), 3623 deletions(-)
 rewrite FinRL_StockTrading_NeurIPS_2018.ipynb (82%)


In [49]:
!git pull

Already up to date.


In [60]:
!git push

Everything up-to-date


<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/FinRL_StockTrading_NeurIPS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)   
* [RLlib Section](#7)            

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [3]:
## install finrl library
# !pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git
!pip install tushare

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-x1j_y9tu
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-x1j_y9tu
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-pmqeaxw4/pyfolio_0d14b79343084638aca68e9a9268bacb
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-pmqeaxw4/pyfolio_0d14b79343084638aca68e9a9268bacb
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-pmqeaxw4/elegantrl_622985889b7b49a58a5eba96c6f7b1df
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-pmqeaxw4/elegantrl_622985889b7b49a58a5eba96c6f7b1df
     |████████████████████████████████| 234 kB 10.6 MB/s 
     |████████████████████████████████| 177 kB 66.5 MB/s 
     |████████████████████████████████| 52.7 MB 160 kB/s 
     |████████████████████████████████| 1.2 MB 58.2 MB/s 
     |████

     |████████████████████████████████| 130 kB 8.2 MB/s 
     |████████████████████████████████| 200 kB 67.0 MB/s 
     |████████████████████████████████| 130 kB 68.9 MB/s 
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.3.2
    Uninstalling websocket-client-1.3.2:
      Successfully uninstalled websocket-client-1.3.2


In [4]:
%cd /content/drive/MyDrive/HYF/Graduation_Fin/FinRL
root='/content/drive/MyDrive/HYF/Graduation_Fin/FinRL'
from finrl import config
from finrl import config_tickers
import os
if not os.path.exists(root+"/" + config.DATA_SAVE_DIR):
    os.makedirs(root+"/" + config.DATA_SAVE_DIR)
if not os.path.exists(root+"/" + config.TRAINED_MODEL_DIR):
    os.makedirs(root+"/" + config.TRAINED_MODEL_DIR)
if not os.path.exists(root+"/" + config.TENSORBOARD_LOG_DIR):
    os.makedirs(root+"/" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists(root+"/" + config.RESULTS_DIR):
    os.makedirs(root+"/" + config.RESULTS_DIR)

/content/drive/MyDrive/HYF/Graduation_Fin/FinRL



<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.tusharedownloader import TushareDownloader
# from finrl.finrl_meta.preprocessor.processor_joinquant import JoinQuantEngineer
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
# from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.finrl_meta.env_stock_trading.env_test import StockTradingEnv

from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools
import jqdatasdk
jqdatasdk.auth('15857500957','Qazwsxedcrfv0957')

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


auth success 


<a id='1.4'></a>
## 2.4. Create Folders

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [6]:
# from config.py start_date is a string
config.TRAIN_START_DATE

'2014-01-01'

In [7]:
# from config.py end_date is a string
config.TEST_END_DATE

'2021-10-01'

In [8]:
df = YahooDownloader(start_date = '2019-01-01',
                     end_date = '2021-10-31',
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
print(config_tickers.DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [10]:
df.shape

(21367, 8)

In [11]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2019-01-02,38.722500,39.712502,38.557499,38.277527,148158800,AAPL,2
1,2019-01-02,192.520004,193.199997,188.949997,174.294479,3009100,AMGN,2
2,2019-01-02,93.910004,96.269997,93.769997,91.087929,4175400,AXP,2
3,2019-01-02,316.190002,323.950012,313.709991,314.645142,3292200,BA,2
4,2019-01-02,124.029999,127.879997,123.000000,116.297020,4783200,CAT,2


In [12]:
ndf = TushareDownloader(start_date = '2019-01-01',
                     end_date = '2021-10-31',
                     ticker_list = ['601398.SH', '600000.SH','601818.SH','600050.SH','300750.SZ']).fetch_data()
ndf

  0%|          | 0/5 [00:00<?, ?it/s]

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


 20%|██        | 1/5 [00:00<00:03,  1.01it/s]

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


 40%|████      | 2/5 [00:01<00:02,  1.23it/s]

...

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


100%|██████████| 5/5 [00:04<00:00,  1.23it/s]


Shape of DataFrame:  (2505, 9)
            date   open   high  close    low      volume  turnover     tic  \
0     2019-10-09  70.01  70.03  69.10  68.38   132218.56      1.10  300750   
1     2019-10-10  70.07  71.64  70.90  69.76   122851.54      1.02  300750   
2     2019-10-11  71.17  72.14  71.46  70.33    94925.62      0.79  300750   
3     2019-10-14  71.90  72.00  70.85  70.50   122035.88      1.01  300750   
4     2019-10-15  70.85  72.10  70.97  70.66    89350.77      0.74  300750   
...          ...    ...    ...    ...    ...         ...       ...     ...   
2500  2021-10-25   4.71   4.73   4.72   4.69  1033424.00      0.04  601398   
2501  2021-10-26   4.72   4.74   4.70   4.69  1187350.00      0.04  601398   
2502  2021-10-27   4.70   4.71   4.70   4.65  1257712.12      0.05  601398   
2503  2021-10-28   4.68   4.70   4.67   4.67   783008.44      0.03  601398   
2504  2021-10-29   4.67   4.69   4.67   4.66  1149808.00      0.04  601398   

      day  
0       2  
1       

/content/drive/MyDrive/HYF/Graduation_Fin/FinRL/finrl/finrl_meta/preprocessor/tusharedownloader.py:65: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop(["price_change","p_change","ma5","ma10","ma20","v_ma5","v_ma10","v_ma20"], 1)


,date,open,high,close,low,volume,turnover,tic,day
0,2019-10-09,70.01,70.03,69.10,68.38,132218.56,1.10,300750,2
1,2019-10-09,11.82,12.13,11.99,11.80,422096.03,0.15,600000,2
2,2019-10-09,6.00,6.07,6.06,5.96,475071.66,0.22,600050,2
3,2019-10-09,5.53,5.60,5.58,5.53,1231203.38,0.05,601398,2
4,2019-10-09,4.07,4.30,4.23,4.06,2877277.25,0.72,601818,2
...,...,...,...,...,...,...,...,...,...
2500,2021-10-29,615.29,645.49,639.22,598.94,202687.02,1.00,300750,4
2501,2021-10-29,8.96,9.00,8.94,8.93,359907.31,0.12,600000,4
2502,2021-10-29,4.04,4.09,4.08,4.04,696518.00,0.23,600050,4
2503,2021-10-29,4.67,4.69,4.67,4.66,1149808.00,0.04,601398,4


In [13]:
ticks=[('601398.XSHG', '600000.XSHG'),
 ('601398.XSHG', '600050.XSHG'),
 ('601398.XSHG', '601818.XSHG'),
 ('600000.XSHG', '600050.XSHG')]
mydf=jqdatasdk.get_price(ticks[0],'2021-01-01','2021-12-31').drop('money',axis=1)
mydf=mydf.rename(columns={'time':'date','code':'tic',})
mydf=mydf.sort_values(['date','tic'],ignore_index=True)
mydf['day']=pd.DataFrame(data=np.zeros(len(mydf),dtype=np.int8))

/usr/local/lib/python3.7/dist-packages/jqdatasdk/api.py:34: PanelObsoleteWarning: 当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口的 panel 参数将固定为 False（0.25 及以上版本的 pandas 不再支持 panel，如使用该数据结构和相关函数请注意修改）
  warnings.warn(PandasChecker.VERSION_NOTICE_MESSAGE, PanelObsoleteWarning)


In [14]:
mydf

,date,tic,open,close,high,low,volume,day
0,2021-01-04,600000.XSHG,9.18,9.22,9.26,9.09,66082052.0,0
1,2021-01-04,601398.XSHG,4.71,4.75,4.75,4.69,373335881.0,0
2,2021-01-05,600000.XSHG,9.21,9.21,9.21,9.06,56577620.0,0
3,2021-01-05,601398.XSHG,4.73,4.71,4.73,4.68,391329348.0,0
4,2021-01-06,600000.XSHG,9.16,9.35,9.36,9.16,65004667.0,0
...,...,...,...,...,...,...,...,...
481,2021-12-29,601398.XSHG,4.66,4.66,4.67,4.64,97664653.0,0
482,2021-12-30,600000.XSHG,8.54,8.54,8.58,8.53,23272484.0,0
483,2021-12-30,601398.XSHG,4.65,4.64,4.66,4.63,141275058.0,0
484,2021-12-31,600000.XSHG,8.54,8.53,8.57,8.53,22929621.0,0


In [15]:
t=mydf
d=np.array(sorted(set(t['date'].dt.date)))
t['day']=t['date'].apply(lambda x:np.where(d==x)[0][0])
mydf=t
mydf

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,date,tic,open,close,high,low,volume,day
0,2021-01-04,600000.XSHG,9.18,9.22,9.26,9.09,66082052.0,0
1,2021-01-04,601398.XSHG,4.71,4.75,4.75,4.69,373335881.0,0
2,2021-01-05,600000.XSHG,9.21,9.21,9.21,9.06,56577620.0,1
3,2021-01-05,601398.XSHG,4.73,4.71,4.73,4.68,391329348.0,1
4,2021-01-06,600000.XSHG,9.16,9.35,9.36,9.16,65004667.0,2
...,...,...,...,...,...,...,...,...
481,2021-12-29,601398.XSHG,4.66,4.66,4.67,4.64,97664653.0,240
482,2021-12-30,600000.XSHG,8.54,8.54,8.58,8.53,23272484.0,241
483,2021-12-30,601398.XSHG,4.65,4.64,4.66,4.63,141275058.0,241
484,2021-12-31,600000.XSHG,8.54,8.53,8.57,8.53,22929621.0,242


In [16]:
mydf=mydf.sort_values(['date','tic'],ignore_index=True)
mydf

,date,tic,open,close,high,low,volume,day
0,2021-01-04,600000.XSHG,9.18,9.22,9.26,9.09,66082052.0,0
1,2021-01-04,601398.XSHG,4.71,4.75,4.75,4.69,373335881.0,0
2,2021-01-05,600000.XSHG,9.21,9.21,9.21,9.06,56577620.0,1
3,2021-01-05,601398.XSHG,4.73,4.71,4.73,4.68,391329348.0,1
4,2021-01-06,600000.XSHG,9.16,9.35,9.36,9.16,65004667.0,2
...,...,...,...,...,...,...,...,...
481,2021-12-29,601398.XSHG,4.66,4.66,4.67,4.64,97664653.0,240
482,2021-12-30,600000.XSHG,8.54,8.54,8.58,8.53,23272484.0,241
483,2021-12-30,601398.XSHG,4.65,4.64,4.66,4.63,141275058.0,241
484,2021-12-31,600000.XSHG,8.54,8.53,8.57,8.53,22929621.0,242


# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: **MACD and RSI.**
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [17]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.INDICATORS,
                    use_vix=True,
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)
# processed = fe.preprocess_data(ndf)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (713, 8)
Successfully added vix


In [18]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [19]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix
0,2019-01-02,AAPL,38.722500,39.712502,38.557499,38.277527,148158800.0,2.0,0.0,41.763168,30.979164,0.0,-66.666667,100.0,38.277527,38.277527,23.219999
1,2019-01-02,AMGN,192.520004,193.199997,188.949997,174.294479,3009100.0,2.0,0.0,41.763168,30.979164,0.0,-66.666667,100.0,174.294479,174.294479,23.219999
2,2019-01-02,AXP,93.910004,96.269997,93.769997,91.087929,4175400.0,2.0,0.0,41.763168,30.979164,0.0,-66.666667,100.0,91.087929,91.087929,23.219999
3,2019-01-02,BA,316.190002,323.950012,313.709991,314.645142,3292200.0,2.0,0.0,41.763168,30.979164,0.0,-66.666667,100.0,314.645142,314.645142,23.219999
4,2019-01-02,CAT,124.029999,127.879997,123.000000,116.297020,4783200.0,2.0,0.0,41.763168,30.979164,0.0,-66.666667,100.0,116.297020,116.297020,23.219999
5,2019-01-02,CRM,133.399994,136.830002,133.050003,135.550003,4783900.0,2.0,0.0,41.763168,30.979164,0.0,-66.666667,100.0,135.550003,135.550003,23.219999
6,2019-01-02,CSCO,42.279999,43.200001,42.209999,38.736618,23833500.0,2.0,0.0,41.763168,30.979164,0.0,-66.666667,100.0,38.736618,38.736618,23.219999
7,2019-01-02,CVX,107.339996,111.680000,107.099998,94.657227,6384800.0,2.0,0.0,41.763168,30.979164,0.0,-66.666667,100.0,94.657227,94.657227,23.219999
8,2019-01-02,DIS,108.099998,109.139999,107.730003,107.654343,9723500.0,2.0,0.0,41.763168,30.979164,0.0,-66.666667,100.0,107.654343,107.654343,23.219999
9,2019-01-02,GS,164.330002,172.250000,163.350006,160.803604,3999400.0,2.0,0.0,41.763168,30.979164,0.0,-66.666667,100.0,160.803604,160.803604,23.219999


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2009-01-01 to 2020-07-01
## Trade data split: 2020-07-01 to 2021-10-31

In [62]:
train = data_split(processed_full, '2009-01-01','2020-07-01')
trade = data_split(processed_full, '2020-07-01','2021-10-31')
print(len(train))
print(len(trade))

10933
9744


In [63]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix
376,2020-06-30,UNH,288.570007,296.450012,287.660004,287.776794,2932900.0,1.0,-0.019475,303.925877,271.251250,52.413064,-25.838437,1.846804,288.020688,281.001441,30.43
376,2020-06-30,V,191.490005,193.750000,190.160004,191.101547,9040100.0,1.0,1.050793,199.130115,185.394818,53.021051,-51.532297,2.013358,191.850757,182.024672,30.43
376,2020-06-30,VZ,54.919998,55.290001,54.360001,50.378475,17414800.0,1.0,-0.437126,53.920283,48.731003,48.097027,-51.018238,8.508886,51.013881,51.466453,30.43
376,2020-06-30,WBA,42.119999,42.580002,41.759998,39.497253,4782100.0,1.0,-0.085024,43.113072,36.918480,48.829914,-14.474052,1.500723,39.597882,39.395699,30.43
376,2020-06-30,WMT,119.220001,120.129997,118.540001,116.543854,6836400.0,1.0,-0.889793,119.908035,113.923057,48.159657,-69.927191,3.847271,118.215775,120.158459,30.43


In [64]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix
0,2020-07-01,AAPL,91.279999,91.839996,90.977501,90.036690,110737200.0,2.0,3.019033,92.835519,80.297759,62.807172,107.489521,29.730532,84.057088,77.831732,28.620001
0,2020-07-01,AMGN,235.520004,256.230011,232.580002,242.083466,6575800.0,2.0,3.665610,234.264605,201.821541,61.279663,271.571212,46.806139,216.584927,217.666553,28.620001
0,2020-07-01,AXP,95.250000,96.959999,93.639999,92.086380,3301000.0,2.0,-0.389163,110.423947,87.759328,48.504828,-66.328701,3.142448,97.244638,90.695525,28.620001
0,2020-07-01,BA,185.880005,190.610001,180.039993,180.320007,49036700.0,2.0,5.443193,220.721139,160.932863,50.925779,24.220608,15.932920,176.472335,155.614168,28.620001
0,2020-07-01,CAT,129.380005,129.399994,125.879997,121.230247,2807800.0,2.0,1.278733,131.250715,113.896725,52.865434,35.572311,14.457404,119.985500,114.191686,28.620001


In [65]:
config.INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [66]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 29, State Space: 291


In [67]:
from finrl.finrl_meta.env_stock_trading.env_test import StockTradingEnv
env_kwargs = { 
        "hmax": 100, 
        "initial_amount": 1000000,
        "num_stock_shares": [0] * stock_dimension, 
        "buy_cost_pct": [0.000675] * stock_dimension, 
        "sell_cost_pct": [0.001] * stock_dimension, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": config.INDICATORS, 
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4 }
e_train_gym = StockTradingEnv(df = train, **env_kwargs)

TypeError: ignored

## Environment for Training



In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [ ]:
agent = DRLAgent(env = env_train)

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC


### Model 1: A2C


In [ ]:
# agent = DRLAgent(env = env_train)
# model_a2c = agent.get_model("a2c")

In [ ]:
# trained_a2c = agent.train_model(model=model_a2c, 
#                              tb_log_name='a2c',
#                              total_timesteps=50000)

### Model 2: DDPG

In [ ]:
# agent = DRLAgent(env = env_train)
# model_ddpg = agent.get_model("ddpg")

In [ ]:
# trained_ddpg = agent.train_model(model=model_ddpg, 
#                              tb_log_name='ddpg',
#                              total_timesteps=50000)

### Model 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

### Model 4: TD3

In [ ]:
# agent = DRLAgent(env = env_train)
# TD3_PARAMS = {"batch_size": 100, 
#               "buffer_size": 1000000, 
#               "learning_rate": 0.001}

# model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

In [ ]:
# trained_td3 = agent.train_model(model=model_td3, 
#                              tb_log_name='td3',
#                              total_timesteps=30000)

### Model 5: SAC

In [ ]:
# agent = DRLAgent(env = env_train)
# SAC_PARAMS = {
#     "batch_size": 128,
#     "buffer_size": 1000000,
#     "learning_rate": 0.0001,
#     "learning_starts": 100,
#     "ent_coef": "auto_0.1",
# }

# model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

In [ ]:
# trained_sac = agent.train_model(model=model_sac, 
#                              tb_log_name='sac',
#                              total_timesteps=60000)

## Trading
Assume that we have $1,000,000 initial capital at 2020-07-01. We use the DDPG model to trade Dow jones 30 stocks.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [ ]:
# data_risk_indicator = processed_full[(processed_full.date<'2020-07-01') & (processed_full.date>='2009-01-01')]
# insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])
# insample_risk_indicator.vix.describe()
# insample_risk_indicator.vix.quantile(0.996)
# insample_risk_indicator.turbulence.describe()
# insample_risk_indicator.turbulence.quantile(0.996)

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [ ]:
#trade = data_split(processed_full, '2020-07-01','2021-10-31')
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv

e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = None,risk_indicator_col='vix', **env_kwargs)
# e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [ ]:
trade.head()

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    # model=trained_a2c, 
    environment = e_trade_gym)

In [ ]:
df_account_value.shape

In [ ]:
df_account_value.tail()

In [ ]:
df_actions.head()

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


In [ ]:
df_account_value.loc[0,'date']

In [ ]:
df_account_value.loc[len(df_account_value)-1,'date']

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("penalty of buying")
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])

In [ ]:
%cd /content/drive/MyDrive/HYF/Graduation_Fin/FinRL

In [ ]:
df_actions.head()